In [25]:
import scanpy as sc
import numpy as np

In [26]:
adata = sc.read("GSE243917_peaksXcells_ALL_raw.h5ad")

In [27]:
adata = adata[adata.obs.ct.isin(["fibroblast","Tcell","Bcell"])]

In [4]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

/wynton/home/corces/vishvak/miniforge3/envs/scverse/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [28]:
cell_type_avg={}

for cell_type in adata.obs.ct.unique():
    cells = adata[adata.obs['ct'] == cell_type]
    avg_counts = np.array(cells.X.mean(axis=0)).flatten()  # Ensure it is a 1D array
    cell_type_avg[cell_type] = avg_counts

In [29]:
cell_types = adata.obs['ct'].unique()
cell_type_avg = {}

for cell_type in cell_types:
    cells = adata[adata.obs['ct'] == cell_type]
    avg_counts = np.array(cells.X.mean(axis=0)).flatten()  # Ensure it is a 1D array
    cell_type_avg[cell_type] = avg_counts

In [30]:
cell_type_avg["fibroblast"]

array([0.07168125, 0.11519598, 0.11711419, ..., 0.04856133, 0.00080767,
       0.19333725], dtype=float32)

In [31]:
def parse_peak(peak):
    chrom, positions = peak.split(':')
    start, end = positions.split('-')
    return chrom, int(start), int(end)

# Create a BedGraph file
for cell_type, avg_counts in cell_type_avg.items():
    with open(f'{cell_type}.bedgraph', 'w') as f:
        # Write the track definition line
        f.write(f'track type=bedGraph name="{cell_type}" description="{cell_type} signal" visibility=full color=200,100,0 altColor=0,100,200 priority=20\n')
        
        for peak_idx, count in enumerate(avg_counts):
            peak = adata.var.index[peak_idx]  # Assuming peaks are in the index
            chrom, start, end = parse_peak(peak)
            f.write(f'{chrom}\t{start}\t{end}\t{count}\n')


In [35]:
import pandas as pd
file_path = 'TCELL_2_covariate_output.txt'  # Replace with your actual file path
data = pd.read_csv(file_path, sep=' ')

# Filter by p-value threshold (e.g., p-value < 0.05)
p_value_threshold = 0.05
filtered_data = data[data['boot_basic_p'] < p_value_threshold]

# Function to parse the peak information
def parse_peak(peak):
    chrom, positions = peak.split(':')
    start, end = positions.split('-')
    return chrom, int(start), int(end)

# Extract chromosome, start, and end from the peak column
filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)

# Select relevant columns for BED file
bed_data = filtered_data[['chrom', 'start', 'end']]

# Save to BED file
bed_file_path = 'Tcell_SCENT.bed'  # Replace with your desired output file path
bed_data.to_csv(bed_file_path, sep='\t', header=False, index=False)

/scratch/vishvak/ipykernel_2745046/1625466632.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/1625466632.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/1625466632.py:16: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [36]:
import pandas as pd
file_path = 'SCENT_2_covariate_B_output/B_2_covariate_output.txt'  # Replace with your actual file path
data = pd.read_csv(file_path, sep=' ')

# Filter by p-value threshold (e.g., p-value < 0.05)
p_value_threshold = 0.05
filtered_data = data[data['boot_basic_p'] < p_value_threshold]

# Function to parse the peak information
def parse_peak(peak):
    chrom, positions = peak.split(':')
    start, end = positions.split('-')
    return chrom, int(start), int(end)

# Extract chromosome, start, and end from the peak column
filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)

# Select relevant columns for BED file
bed_data = filtered_data[['chrom', 'start', 'end']]

# Save to BED file
bed_file_path = 'Bcell_SCENT.bed'  # Replace with your desired output file path
bed_data.to_csv(bed_file_path, sep='\t', header=False, index=False)

/scratch/vishvak/ipykernel_2745046/3983426034.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/3983426034.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/3983426034.py:16: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [37]:
import pandas as pd
file_path = 'SCENT_2_covariate_output/fibroblast_2_covariate_output.txt'  # Replace with your actual file path
data = pd.read_csv(file_path, sep=' ')

# Filter by p-value threshold (e.g., p-value < 0.05)
p_value_threshold = 0.05
filtered_data = data[data['boot_basic_p'] < p_value_threshold]

# Function to parse the peak information
def parse_peak(peak):
    chrom, positions = peak.split(':')
    start, end = positions.split('-')
    return chrom, int(start), int(end)

# Extract chromosome, start, and end from the peak column
filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)

# Select relevant columns for BED file
bed_data = filtered_data[['chrom', 'start', 'end']]

# Save to BED file
bed_file_path = 'Fibroblast_SCENT.bed'  # Replace with your desired output file path
bed_data.to_csv(bed_file_path, sep='\t', header=False, index=False)

/scratch/vishvak/ipykernel_2745046/1390567506.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/1390567506.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['chrom', 'start', 'end']] = filtered_data['peak'].apply(parse_peak).apply(pd.Series)
/scratch/vishvak/ipykernel_2745046/1390567506.py:16: SettingWithCopyWarning: 
A value is trying to be set on a cop